In [ ]:
PROFILE='render_notebook'
MODEL_NAME='probabilities-example'


## Illustration of predict_proba

based on: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB

In [1]:
import numpy as np
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
Y = np.array([1, 1, 1, 2, 2, 2])

In [2]:
from sklearn.naive_bayes import GaussianNB

In [3]:
clf = GaussianNB()
clf.fit(X, Y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [4]:
clf.predict([[-0.8, -1]])

array([1])

In [5]:
clf.predict_proba([[-0.8, -1]])

array([[9.99999949e-01, 5.05653254e-08]])

## Save and upload model

In [6]:

from waylay import WaylayClient, RestResponseError
waylay = WaylayClient.from_profile()
waylay.byoml.model.upload(
    model_name=MODEL_NAME, 
    trained_model=clf, 
    framework='sklearn',
    description='Demo model sklearn probabilities.'
)

{'message': 'Model successfully uploaded'}

In [7]:
waylay.byoml.model.get(MODEL_NAME)

{'createdBy': 'users/bd87db46-e8fc-4186-bdf4-a8f9478ed0d3',
 'createdOn': '2021-06-02T15:08:56.303093+00:00',
 'description': 'Demo model sklearn probabilities.',
 'framework': 'sklearn',
 'name': 'probabilities-example',
 'supported_endpoints': ['predict'],
 'updatedBy': 'users/bd87db46-e8fc-4186-bdf4-a8f9478ed0d3',
 'updatedOn': '2021-06-02T15:08:56.303130+00:00'}

## Check if model and uploaded model act the same

In [8]:
features = [[-0.8, -1]]

In [9]:
clf.predict(features)

array([1])

In [10]:
result= waylay.byoml.model.predict(MODEL_NAME, features)

result

array([1])

In [11]:
clf.predict_proba(features)

array([[9.99999949e-01, 5.05653254e-08]])

In [12]:
## TODO: custom body parameters not supported yet (https://github.com/waylayio/waylay-py/issues/77)

# waylay.byoml.model.predict(MODEL_NAME, features, body={'probabilities': True})

## workaround
from waylay.service import WaylayResource
class CustomModelResource(WaylayResource):
    actions = {
        'predict': {
            'method': 'POST',
            'url': '/models/{}/predict'
        }
    }
waylay.byoml._add_waylay_resource('model_x', CustomModelResource)
waylay.byoml.reconfigure()

result = waylay.byoml.model_x.predict(model_name, body={'instances': features, 'probabilities': True})
result.body['predictions']

[[0.9999999494346747, 5.056532536795868e-08]]

## Delete model

In [13]:
waylay.byoml.model.remove(MODEL_NAME)

{'message': 'Model successfully deleted'}